In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

// TODO(Johann) could these be moved to json configuration files?
val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val IMPORT_CONFIGURATION_FILE = "../../data/provision/import-configuration.json"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-11.12"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=ceb096c1-5a67-4617-9a35-a8c6992d2ac9, username=Johann-11.12)

In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=3a69408c-36b7-45fb-a4f6-cd477560bf6f, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=ef298e6d-3956-4d8b-8528-4d9ac5165982, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=d5e437e7-ed8b-4345-bbb2-508123fee19c, name=ING RON, unit=Currency(value=RON))
AccountTO(id=d261dc85-88ab-47a5-9cb4-e75cc64956c2, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=413502ac-7edb-4a77-802c-0b6f9cd54937, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=2238c135-17a2-4084-a874-6831bcb50ab1, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=e809824f-54b7-485b-af85-09016601b751, name=Other, unit=Currency(value=RON))
AccountTO(id=d4d9963d-b5cf-4d90-8fe7-e0f72e96f2b5, name=BT RON, unit=Currency(value=RON))
AccountTO(id=337d31a9-afd0-40a5-84e6-78691948baae, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=2da9655a-8274-48d9-97f0-b2c4fb57bace, name=Expenses)
FundTO(id=282c5898-9756-41bc-881e-b375b91267ca, name=Work Income)
FundTO(id=7465fe9b-55be-4b48-9358-4ccd9c322b30, name=Savings)
FundTO(id=5a29591d-57e6-4570-be54-a0bddd862b08, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawJson: String = File(IMPORT_CONFIGURATION_FILE).readText()
val jsonFormat = Json { ignoreUnknownKeys = true }
val importConfiguration = jsonFormat.decodeFromString<ImportConfigurationTO>(dataConfigurationRawJson)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles = File("../../data/wallet/2019")
    .listFiles()?.toList() ?: error("no files found")
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=ba0abfc6-1ee4-4828-a563-a196e34f740a, status=COMPLETED, reason=null)